In [37]:
# imports
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import googlemaps
import folium
import pyproj

In [38]:
df = pd.read_csv('C:/Users/ashly/Documents/Hackathons/DataForGood/Team-4/ACSST5Y2022.S2201-Data.csv')
df.head()

C:\Users\ashly\AppData\Local\Temp\ipykernel_26072\1694890895.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,70,71,72,73,74,75,76,77,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,226,227,228,229,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,374,375,376,377,378,379,380,381) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/

,GEO_ID,NAME,S2201_C01_001E,S2201_C01_001M,S2201_C01_002E,S2201_C01_002M,S2201_C01_003E,S2201_C01_003M,S2201_C01_004E,S2201_C01_004M,...,S2201_C06_034M,S2201_C06_035E,S2201_C06_035M,S2201_C06_036E,S2201_C06_036M,S2201_C06_037E,S2201_C06_037M,S2201_C06_038E,S2201_C06_038M,Unnamed: 458
0,Geography,Geographic Area Name,Estimate!!Total!!Households,Margin of Error!!Total!!Households,Estimate!!Total!!Households!!With one or more ...,Margin of Error!!Total!!Households!!With one o...,Estimate!!Total!!Households!!No people in the ...,Margin of Error!!Total!!Households!!No people ...,Estimate!!Total!!Households!!Married-couple fa...,Margin of Error!!Total!!Households!!Married-co...,...,Margin of Error!!Percent households not receiv...,Estimate!!Percent households not receiving foo...,Margin of Error!!Percent households not receiv...,Estimate!!Percent households not receiving foo...,Margin of Error!!Percent households not receiv...,Estimate!!Percent households not receiving foo...,Margin of Error!!Percent households not receiv...,Estimate!!Percent households not receiving foo...,Margin of Error!!Percent households not receiv...,NaN
1,1400000US01001020100,Census Tract 201; Autauga County; Alabama,700,114,323,74,377,114,444,125,...,(X),(X),(X),19.2,12.5,24.5,10.7,56.3,15.4,NaN
2,1400000US01001020200,Census Tract 202; Autauga County; Alabama,544,101,298,85,246,62,154,41,...,(X),(X),(X),7.2,8.1,38.6,20.3,54.3,18.8,NaN
3,1400000US01001020300,Census Tract 203; Autauga County; Alabama,1305,227,565,166,740,189,569,192,...,(X),(X),(X),10.1,5.7,28.6,12.0,61.2,13.3,NaN
4,1400000US01001020400,Census Tract 204; Autauga County; Alabama,1666,219,948,179,718,165,1048,212,...,(X),(X),(X),27.1,11.1,34.4,11.3,38.5,10.2,NaN


- Each GEOID contains 14..US at the start, ignore '1400000US' for each
dont use 155 - 306
- Important Columns: (majority minority area for a prediction of where we should help and where majority of youth are)
0. GEO_ID 
.. S2201_C01_001E - Total Households
1. S2201_C01_009E - With Children under 18
2. S2201_C01_021E - Below Poverty Status in the last 12 months
3. S2201_C01_022E - At or Above Poverty Status in the last 12 months
4. S2201_C01_023E - One or more people with disability in household
5. S2201_C01_026E - Black Alone
6. S2201_C01_027E - American Indian Alone
7. S2201_C01_028E - Asian Alone
8. S2201_C01_032E - Hispanic Alone
9. S2201_C01_029E - Native Hawaiian Alone
10. S2201_C01_035E - Family Work Status
11. S2201_C01_036E - No work status in the last 12 months
12. S2201_C01_037E - 1 work status in the last 12 months
13. S2201_C01_038E - 2 work status' in the last 12 months
14. S2201_C02_001E - Percent of Households
15. S2201_C02_012E - Single Parent (Male)
16. S2201_C02_013E - Single Parent (Female)
17. S2201_C02_014E - Non-Family Household with children (concerning stat)

In [39]:
# find the names of each Philly value
philly_names = []


for name in df['NAME'].unique():
    if 'Philadelphia' in name:
        philly_names.append(name)
        
df = df[df['NAME'].isin(philly_names)]
df.head()

,GEO_ID,NAME,S2201_C01_001E,S2201_C01_001M,S2201_C01_002E,S2201_C01_002M,S2201_C01_003E,S2201_C01_003M,S2201_C01_004E,S2201_C01_004M,...,S2201_C06_034M,S2201_C06_035E,S2201_C06_035M,S2201_C06_036E,S2201_C06_036M,S2201_C06_037E,S2201_C06_037M,S2201_C06_038E,S2201_C06_038M,Unnamed: 458
65977,1400000US42101000101,Census Tract 1.01; Philadelphia County; Pennsy...,1413,223,201,118,1212,234,268,111,...,(X),(X),(X),4.9,8.2,12.0,11.0,83.0,13.9,NaN
65978,1400000US42101000102,Census Tract 1.02; Philadelphia County; Pennsy...,1960,354,265,84,1695,350,359,129,...,(X),(X),(X),6.7,8.2,11.3,9.6,82.0,14.2,NaN
65979,1400000US42101000200,Census Tract 2; Philadelphia County; Pennsylvania,1686,431,412,127,1274,444,474,119,...,(X),(X),(X),12.9,12.2,2.6,4.3,84.5,11.2,NaN
65980,1400000US42101000300,Census Tract 3; Philadelphia County; Pennsylvania,2347,374,923,317,1424,201,527,144,...,(X),(X),(X),17.5,12.5,26.7,13.2,55.9,13.7,NaN
65981,1400000US42101000401,Census Tract 4.01; Philadelphia County; Pennsy...,1957,316,511,195,1446,298,331,168,...,(X),(X),(X),0.0,10.3,46.9,26.3,53.1,26.3,NaN


In [41]:
# Utilize the geocoder for converting addresses 
geolocator = Nominatim(user_agent="my_geocoder")

## find the location of Hunting Park 
location = "Hunting Park, Philadelphia, PA 19140"
geocode = geolocator.geocode(location)

# Extract latitude and longitude
if geocode:
    latitude = geocode.latitude
    longitude = geocode.longitude
    print("Latitude:", latitude)
    print("Longitude:", longitude)
else:
    print("Location not found.")

m = folium.Map(location=[latitude, longitude], zoom_start=20) # Hunting Park
zipcodes_geojson = 'C:/Users/ashly/Documents/Hackathons/DataForGood/Team-4/GeoMap/pa_pennsylvania_zip_codes_geo.min.json'
folium.GeoJson(zipcodes_geojson).add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)  

temp_map = {}




## find the lat and lon for placing onto the map
df['GEO_ID'] = [g[9:] for g in df['GEO_ID']]
df_zipcode = pd.read_csv('C:/Users/ashly/Documents/Hackathons/DataForGood/Team-4/GeoMap/zip_lat_long.csv')
gdf = gpd.read_file('C:/Users/ashly/Documents/Hackathons/DataForGood/tl_2023_42_tract/tl_2023_42_tract.shp')
gdf = gdf.rename(columns={'GEOID': 'GEO_ID'}) # match the GEO ID NAMES

# merge the gdf and df on GEOID and GEO_ID
df_merged_temp = pd.merge(df, gdf, on='GEO_ID')
df_zipcode.columns = ['zipcode', 'INTPTLAT', 'INTPTLON']
df_zipcode['INTPTLAT'] = [str(value) for value in df_zipcode['INTPTLAT']]
df_zipcode['INTPTLON'] = [str(value) for value in df_zipcode['INTPTLON']]

df_merged = pd.merge(df_merged_temp, df_zipcode, on=['INTPTLAT', 'INTPTLON'], how='outer')
print(df_merged.head())
## view the zip code and lat lon dataset

## finding the metrics to be displayed on the map
rate_below_poverty = []
rate_black_alone = []
rate_no_work = []
rate_total_children = []
score_list = []
total_households = 'S2201_C01_001E'
for index, row in df_merged.iterrows():
    if row[total_households] != 0:
        rate_below_poverty.append(row['S2201_C01_021E'] / row[total_households])
        rate_black_alone.append(row['S2201_C01_026E'] / row[total_households])
        rate_no_work.append(row['S2201_C01_036E'] / row[total_households])
        rate_total_children.append(row['S2201_C01_009E'] / row[total_households])
        marginalized_group = (row['S2201_C01_026E'] / row[total_households]) + (row['S2201_C01_027E'] / row[total_households]) + (row['S2201_C01_028E'] / (row[total_households]) + (row['S2201_C01_032E'] )/ row[total_households]) + (row['S2201_C01_029E'] / row[total_households])
        score = (0.50*(row['S2201_C01_021E'] / row[total_households]) + 0.30*(marginalized_group) + 0.20*(row['S2201_C01_009E'] / row[total_households]))
        score_list.append(score)
        
    else:
        ## cannot dropna due to the format of the census data and overlapping of 0 values
        rate_below_poverty.append(0)
        rate_black_alone.append(0)
        rate_no_work.append(0)
        rate_total_children.append(0)
        score_list.append(0)
        
        
# temp_map['rate_below_poverty'] = rate_below_poverty
# temp_map['rate_black_alone'] = rate_black_alone
# temp_map['rate_no_work'] = rate_no_work
temp_map['IE_SCORE'] = score_list
temp_map['Latitude'] = df_merged['INTPTLAT']    
temp_map['Longitude'] = df_merged['INTPTLON']
temp_map['GEO_ID'] = df_merged['GEO_ID']
temp_map['ZIP_CODE'] = df_merged['zipcode']
                    
df_map = pd.DataFrame(temp_map)
df_map
# heat_data = []
# for index, row in df_map.iterrows(): 
#     heat_data.append([row['Latitude'], row['Longitude'], row['IE_SCORE']])


# HeatMap(heat_data).add_to(m)

# m.save('C:/Users/ashly/Documents/Hackathons/DataForGood/Team-4/GeoMap/score_map.html')
    

Latitude: 40.016501
Longitude: -75.1437876
  GEO_ID NAME_x S2201_C01_001E S2201_C01_001M S2201_C01_002E S2201_C01_002M  \
0    NaN    NaN            NaN            NaN            NaN            NaN   
1    NaN    NaN            NaN            NaN            NaN            NaN   
2    NaN    NaN            NaN            NaN            NaN            NaN   
3    NaN    NaN            NaN            NaN            NaN            NaN   
4    NaN    NaN            NaN            NaN            NaN            NaN   

  S2201_C01_003E S2201_C01_003M S2201_C01_004E S2201_C01_004M  ... NAME_y  \
0            NaN            NaN            NaN            NaN  ...    NaN   
1            NaN            NaN            NaN            NaN  ...    NaN   
2            NaN            NaN            NaN            NaN  ...    NaN   
3            NaN            NaN            NaN            NaN  ...    NaN   
4            NaN            NaN            NaN            NaN  ...    NaN   

  NAMELSAD MTFCC FU

,IE_SCORE,Latitude,Longitude,GEO_ID,ZIP_CODE
0,NaN,13.260372,144.700671,NaN,96916
1,NaN,13.300206,144.740075,NaN,96917
2,NaN,13.377481,144.702278,NaN,96915
3,NaN,13.384537,144.661298,NaN,96928
4,NaN,13.450553,144.75071,NaN,96910
...,...,...,...,...,...
33139,NaN,70.118416,-143.686957,NaN,99747
33140,NaN,70.428206,-149.287534,NaN,99734
33141,NaN,70.468398,-156.795478,NaN,99791
33142,NaN,70.645765,-159.898711,NaN,99782
